In [1]:
import networkx as nx

import numpy as np
import pandas as pd

from scipy.spatial import distance_matrix

from bokeh.io import show, output_file
from bokeh.models import (Plot, Range1d, MultiLine, Circle, TapTool, OpenURL, HoverTool, CustomJS, 
                         Slider, Column, StaticLayoutProvider)
from bokeh.plotting import figure, from_networkx
from bokeh.palettes import Spectral4

from matplotlib import cm
from matplotlib.colors import to_hex

from bokeh.models import FixedTicker, LinearColorMapper, LogColorMapper, ColorBar, BasicTicker, LogTicker

from tqdm.notebook import tqdm

In [2]:
d_matrix = np.loadtxt('output/test_1_edges', delimiter=',')
# d_matrix = np.loadtxt('output/jones13n_20_edges', delimiter=',')
d_matrix

array([[ 0.,  9., 10., 18., 21.],
       [ 9.,  0.,  2., 18., 20.],
       [10.,  2.,  0., 18., 18.],
       [18., 18., 18.,  0.,  3.],
       [21., 20., 18.,  3.,  0.]])

In [3]:
d_matrix.max()

21.0

In [4]:
distance_from_landmarks = np.loadtxt('output/test_1_distance_of_points_in_order_from_landmarks', delimiter=',')
# distance_from_landmarks = np.loadtxt('output/jones13n_20_distance_of_points_in_order_from_landmarks', delimiter=',')

distance_from_landmarks

array([[ 0.,  1., 10., 12., 30., 33.],
       [ 0.,  2.,  9., 10., 20., 23.],
       [ 0.,  2., 11., 12., 18., 21.],
       [ 0.,  3., 18., 20., 29., 30.],
       [ 0.,  3., 21., 23., 32., 33.]])

In [5]:
color_of_landmarks = np.loadtxt('output/test_1_coloration_in_order_from_landmarks', delimiter=',')
# color_of_landmarks = np.loadtxt('output/jones13n_20_coloration_in_order_from_landmarks', delimiter=',')
color_of_landmarks

array([[1. , 1.5, 2. , 2.5, 3. , 3.5],
       [3. , 3.5, 3. , 2.5, 3. , 3.5],
       [4. , 3.5, 3. , 2.5, 3. , 3.5],
       [5. , 5.5, 5. , 4.5, 4. , 3.5],
       [6. , 5.5, 5. , 4.5, 4. , 3.5]])

In [6]:
def create_color_matrix(color_of_landmarks, my_palette):
    MAX_VALUE = color_of_landmarks.max()
    MIN_VALUE = color_of_landmarks.min()
    
    color_matrix = []
    
    for row in tqdm(color_of_landmarks):
        color_matrix.append([ to_hex(my_palette((value - MIN_VALUE) / (MAX_VALUE - MIN_VALUE))) for value in row ])
        
    return color_matrix


In [7]:
my_palette = cm.get_cmap(name='jet') # multicolor
color_matrix = create_color_matrix(color_of_landmarks, my_palette)
color_matrix

  0%|          | 0/5 [00:00<?, ?it/s]

[['#000080', '#0000f1', '#004cff', '#00b0ff', '#29ffce', '#7dff7a'],
 ['#29ffce', '#7dff7a', '#29ffce', '#00b0ff', '#29ffce', '#7dff7a'],
 ['#ceff29', '#7dff7a', '#29ffce', '#00b0ff', '#29ffce', '#7dff7a'],
 ['#ff6800', '#f10800', '#ff6800', '#ffc400', '#ceff29', '#7dff7a'],
 ['#800000', '#f10800', '#ff6800', '#ffc400', '#ceff29', '#7dff7a']]

In [8]:
# define the coordinate of the landmarks
# my_landmarks = [[0,0],[0,1],[2,0],[3,3]]
# d_matrix = distance_matrix(my_landmarks, my_landmarks)
G = nx.from_numpy_matrix(d_matrix)

In [11]:
plot = Plot(plot_width=700, plot_height=700,
            x_range=Range1d(-2, 2), y_range=Range1d(-2, 2))
plot.toolbar_location = None

NODE_SIZE = 20

min_epsilon = 0
max_epsilon = distance_from_landmarks.max()

for edge in G.edges:
    G.edges[edge]['alpha'] = 0
    
for i, node in enumerate(G.nodes):
    G.nodes[node]['size_scale'] = NODE_SIZE
    G.nodes[node]['size'] = NODE_SIZE
    G.nodes[node]['distance_from_landmark'] = distance_from_landmarks[i].tolist()
    G.nodes[node]['color_list'] = color_matrix[i]
    G.nodes[node]['color'] = color_matrix[i][0]

graph_renderer = from_networkx(G, nx.kamada_kawai_layout, scale=1, center=(0, 0))

################################
# If your landmarks are in the plane use the following two lines to plot them
# in their exaxt location
# if they are in higher dimension comment them to keep using the kamada_kawai_layout 
#
# convert the my_landmarks lists into a dictionary of 2D-coordinates
# and assign each entry to a node on the ``node_indices`` list
#graph_layout = dict(zip(list(range(len(my_landmarks))), my_landmarks))

# use the provider model to supply coourdinates to the graph
#graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)
################################

graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='color', fill_alpha=0.5)
graph_renderer.edge_renderer.glyph = MultiLine(line_color = "black", line_alpha = 'alpha', line_width = 2)
plot.renderers.append(graph_renderer)

code = """ 
    var current_epsilon = cb_obj.value;
    
    var node_data = graph_renderer.node_renderer.data_source.data;
    var edge_data = graph_renderer.edge_renderer.data_source.data;
    
    for (var i = 0; i < node_data['size'].length; i++) {
        var counter = 0;
        for (var j = 0; j < node_data['distance_from_landmark'][i].length; j++) {
            if (node_data['distance_from_landmark'][i][j]  > current_epsilon) { break; }
            counter++;
        }

        //graph_renderer.node_renderer.data_source.data['size'][i] = Math.max(10,Math.min(Math.sqrt(counter*node_data['size_scale'][i]), 100));
        graph_renderer.node_renderer.data_source.data['size'][i] = counter*node_data['size_scale'][i];
        graph_renderer.node_renderer.data_source.data['color'][i] = graph_renderer.node_renderer.data_source.data['color_list'][i][counter-1];
    }
    
    for (var i = 0; i < edge_data['alpha'].length; i++) {
        graph_renderer.edge_renderer.data_source.data['alpha'][i] = + (edge_data['weight'][i] <= current_epsilon);
    }
    
    graph_renderer.node_renderer.data_source.change.emit();
    graph_renderer.edge_renderer.data_source.change.emit();
    
    
"""
callback = CustomJS(args = dict(graph_renderer = graph_renderer),
                    code = code)
slider = Slider(start=min_epsilon, end=max_epsilon, step=0.1, value=min_epsilon,
               title='epsilon')
slider.js_on_change('value', callback)

# # continuous colorbar
num_ticks = 100
low = color_of_landmarks.min()
high = color_of_landmarks.max()

color_mapper = LinearColorMapper(palette=[to_hex(my_palette(color_id)) 
                                          for color_id in np.linspace(0, 1, num_ticks)], 
                                 low=low, high=high)

color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size='14pt',
                     label_standoff=12,
                    )

plot.add_layout(color_bar, 'right')

layout = Column(plot, slider)

In [12]:
show(layout)